# **RIC1-CRDM Descriptives**

Input: 
* CRDM behavioral file from Psychopy [sub-####_ses-crdm_task.csv]

Ouput: 
* CRDM CSV file [sub-####_ses-crdm.csv]
* CRDM Choice Plots [sub-####_ses-crdm_choice-plots.png]
* CRDM Risk Figures [sub-####_ses-crdm_risk-trials.png]
* CRDM Ambiguity Trial Figures [sub-####_ses-crdm_amb-trials.png]

In [2]:
import os
import os.path, sys
import numpy as np
import pandas as pd
from glob import glob
%matplotlib inline
import seaborn as sns
sns.set_theme(style = "white", palette = "muted")
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
analysis = "1"
base_proj_dir = "Z:/data/RIC" ## base project directory
data_dir = "Z:/data/RIC/sourcedata/RIC1{0}".format(analysis) ## directory containing data


subs        = []
amb_levels  =  [0, 24, 50, 74]
prob_levels =  [13, 25, 38, 50, 75]
prob_counts =  {"13p_0a_safe":[], "13p_0a_lott":[], "13p_0a_nr":[],
                "25p_0a_safe":[], "25p_0a_lott":[], "25p_0a_nr":[],
                "38p_0a_safe":[], "38p_0a_lott":[], "38p_0a_nr":[],
                "50p_0a_safe":[], "50p_0a_lott":[], "50p_0a_nr":[],
                "75p_0a_safe":[], "75p_0a_lott":[], "75p_0a_nr":[]}
amb_counts  =  {"50p_0a_safe":[],  "50p_0a_lott":[],  "50p_0a_nr":[],
                "50p_24a_safe":[], "50p_24a_lott":[], "50p_24a_nr":[],
                "50p_50a_safe":[], "50p_50a_lott":[], "50p_50a_nr":[],
                "50p_74a_safe":[], "50p_74a_lott":[], "50p_74a_nr":[]} ## across subjects counts for ambiguity trials

sub_files = sorted(glob(os.path.join(data_dir, "23_IDM_*.csv"))) ## grab all CRDM task csvs files in folder
for i, curr_file in enumerate(sub_files): ## go through each subject file
    subs.append(os.path.basename(curr_file)[7:11]) ## append current subject ID to subs list
    sub_cols = ["prob", "amb", "sure_amt", "lott_amt", "choice"] ## trial variables to be saved to output file
    sub_df = pd.DataFrame(columns = sub_cols) ## subject-specific dataframe   
    raw_df = pd.read_csv(curr_file) ## import Ss behav file into Panda's dataframepd.read_csv(curr_file) #import Ss behav file into Panda's dataframe
    resp_df = raw_df.loc[(raw_df["crdm_trial_type"] == "task") & (raw_df["crdm_choice"].notnull())] ## only task trials with responses
    nonresp_df = raw_df.loc[(raw_df["crdm_trial_type"] == "task") & (raw_df["crdm_choice"].isnull())] ## only task trials with nonresponses (important for creating EV file)

    for k, this_prob in enumerate(prob_levels):
        for l, this_choice in enumerate(["safe", "lott"]):
            prob_counts["{0}p_0a_{1}".format(this_prob, this_choice)].append(len(resp_df.loc[(resp_df["crdm_lott_p"] == this_prob) 
                                                                                            & (resp_df["crdm_amb_lev"] == 0) 
                                                                                            & (resp_df["crdm_choice"] == l)]))
        prob_counts["{0}p_0a_nr".format(this_prob)].append(len(nonresp_df.loc[(nonresp_df["crdm_lott_p"] == this_prob) 
                                                                            & (nonresp_df["crdm_amb_lev"] == 0)]))
    for k, this_amb in enumerate(amb_levels): ## calculates and saves ambiguity trial count info for response and nonresponse trials
        for l, this_choice in enumerate(["safe", "lott"]):
            amb_counts["50p_{0}a_{1}".format(this_amb, this_choice)].append(len(resp_df.loc[(resp_df["crdm_lott_p"] == 50)
                                                                                            & (resp_df["crdm_amb_lev"] == this_amb) 
                                                                                            & (resp_df["crdm_choice"] == l)]))
        amb_counts["50p_{0}a_nr".format(this_amb)].append(len(nonresp_df.loc[(nonresp_df["crdm_lott_p"] == 50) 
                                                                            & (nonresp_df["crdm_amb_lev"] == this_amb)]))
    temp_df = pd.DataFrame() ## create fresh DF for each pain lev/prob combo
    temp_df['prob'] = resp_df['crdm_lott_p'] ## trial probability
    temp_df['amb'] = resp_df['crdm_amb_lev'] ## trial ambiguity
    temp_df['sure_amt'] = resp_df['crdm_sure_amt'] ## trial certain amount
    temp_df['lott_amt'] = resp_df['crdm_lott_top'] + resp_df['crdm_lott_bot'] ## trial lottery amount
    temp_df['choice'] = resp_df['crdm_choice'] ## trial choice (-1 = nonresponse, 0 = certain, 1 = lottery)
    temp_df = temp_df.astype(int) ## convert elements of dataframe to integers

    temp_df2 = pd.DataFrame() ## create separate DF for nonresponse task trials
    temp_df2["prob"] = nonresp_df["crdm_lott_p"] ## lottery probability
    temp_df2["amb"] = nonresp_df["crdm_amb_lev"] ## lottery ambiguity
    temp_df2["sure_amt"] = nonresp_df["crdm_sure_amt"] ## certain $ amount
    temp_df2["lott_amt"] = nonresp_df["crdm_lott_top"] + nonresp_df["crdm_lott_bot"] ## winning lottery amount
    temp_df2["choice"] = [-1] * len(temp_df2["sure_amt"]) ## trial choice [all -1 to indicate nonresponse]
    temp_df2 = temp_df2.astype(int) ## convert all elements of dataframe to integers


    ##### PROBABILITY TRIALS #####
    fig1 = plt.figure(i, figsize = (14, 8)) ## create Ss figure
    for prob_idx, curr_prob in enumerate(prob_levels): ## 5 probability levels
        prob_loc = 1 + prob_idx ## index for sub-plot location
        fig1.add_subplot(2, 5, prob_loc) ## defining location of subfigure
        ## probability specific dataframe (individual trials)
        curr_trials1 = temp_df.loc[(temp_df["prob"] == curr_prob) & (temp_df["amb"] == 0)] ## grab trials for current probability level
        sub_df = pd.concat([sub_df, curr_trials1], ignore_index=True) ## add to subject output file dataframe
        ## isolation of unique lottery/choice pairs 
        all_x = curr_trials1["lott_amt"].tolist() ## convert all lottery amounts series to list
        all_y = curr_trials1["choice"].tolist() ## convert all choices series to list
        amt_choice_combos = list(zip(all_x, all_y)) ## tuples of lottery/choices for all relevant trials
        unique_combos = sorted(list(set(amt_choice_combos))) ## only unique lottery/choice combos
        #combo_ct = [amt_choice_combos.count(item) for item in unique_combos] ## number of trials for each element of unique_combos
        x, y = list(zip(*sorted(unique_combos))) ## unzip unique_combos
        x2 = []
        for curr_x in x: ## add $ to x-axis values
            x2.append("$"+str(curr_x))
        plt.plot(x2, y, 'ro-') ## red = risk trials
        plt.ylim([-0.05, 1.05]) ## buffer extremes of y-axis
        sns.despine(top = True) ## remove ticks and upper/right sides of subplots
        if prob_idx == 0: ## only add y axis labels to far left subfigures (row)
            plt.yticks([0, 1], ["Certain (0)", "Lottery (1)"])
            plt.ylabel("Probability of Lottery Choice")
        else:
            plt.yticks([])
        plt.title(str(curr_prob) + "% Prob - 0% Amb", fontsize = 12, color = "red") ## title for each probability subplot


    ##### AMBIGUITY TRIALS #####
    for amb_idx, curr_amb in enumerate(amb_levels):
        amb_loc = 5 + amb_idx
        fig1.add_subplot(2, 4, amb_loc)
        curr_trials2 = temp_df.loc[(temp_df["prob"] == 50) & (temp_df["amb"] == curr_amb)] 
        all_x = curr_trials2["lott_amt"].tolist() 
        all_y = curr_trials2["choice"].tolist() 
        amt_choice_combos = list(zip(all_x, all_y)) 
        unique_combos = list(set(amt_choice_combos)) 
        #combo_ct = [amt_choice_combos.count(item) for item in unique_combos] 
        x, y = list(zip(*sorted(unique_combos)))
        x2 = []
        for curr_x in x:
            x2.append("$"+str(curr_x))
        curr_trials3 = curr_trials2.loc[(curr_trials2["amb"] != 0)] ## used specifically to prevent duplication in output csv
        sub_df = pd.concat([sub_df, curr_trials3], ignore_index=True) ## add to Ss csv dataframe
        plt.plot(x2, y, 'bo-') ## blue = ambiguity trials
        plt.ylim([-0.05, 1.05])
        sns.despine(top=True)
        if amb_idx == 0:
            plt.yticks([0, 1], ["Certain (0)", "Lottery (1)"])
            plt.ylabel("Probability of Lottery Choice")
        else:
            plt.yticks([])
        plt.xlabel("Lottery Value")
        plt.title("50% Prob - {0}% Amb".format(str(curr_amb)), fontsize = 12, color = "blue")
    ## Complete plot
    plt.suptitle("RIC{0} sub-{1}".format(analysis, subs[i]), fontsize = 18) 
    save_sub_dir = os.path.join(base_proj_dir, "derivatives/RIC{0}/descriptives/crdm/sub-{1}".format(analysis, subs[i])) ## set Ss specific save directory
    if not os.path.exists(save_sub_dir): ## new Ss
        os.makedirs(save_sub_dir) ## make new Ss save directory
        print("---------------> NEW PARTICIPANT DIRECTORY CREATED: sub-{0}".format(subs[i])) ## indicate new Ss directory was made
    fig_save = os.path.join(save_sub_dir, "sub-{0}_ric{1}-crdm_choice-plots.png".format(subs[i]), analysis) 
    print("Saving to: {0}".format(fig_save)) 
    plt.savefig(fig_save) 
    #plt.show() 
    sub_filename = os.path.join(save_sub_dir, "sub-{0}_ric{1}-crdm_choice-behavior.csv".format(subs[i]), analysis) ## path/filename to save csv
    print("Saving to: {0}".format(sub_filename)) ## indicate Ss save directory
    sub_df.to_csv(sub_filename, index = False) ## save output file

prob_counts_df = pd.DataFrame(prob_counts, index = subs) ## convert prob_counts dict to DF
amb_counts_df = pd.DataFrame(amb_counts, index = subs) ## convert amb_counts dict to DF

# Choice Counts & Percentages

## Risky Trials

In [ ]:
for idx, curr_sub in enumerate(subs):
    risk_count =   {"13p_safe":[], "13p_lott":[], "13p_nr":[],
                    "25p_safe":[], "25p_lott":[], "25p_nr":[],
                    "38p_safe":[], "38p_lott":[], "38p_nr":[],
                    "50p_safe":[], "50p_lott":[], "50p_nr":[],
                    "75p_safe":[], "75p_lott":[], "75p_nr":[]}
    risk_prop = {"13p_lott":[], "25p_lott":[], "38p_lott":[], "50p_lott":[], "75p_lott":[]}
    for x in ["13", "25", "38", "50", "75"]:
        for y in ["safe", "lott", "nr"]:
            risk_count["{0}p_{1}".format(x, y)].append(prob_counts_df["{0}p_0a_{1}".format(x, y)][idx]) 
        risk_prop["{0}p_lott".format(x)].append(prob_counts_df["{0}p_0a_lott".format(x)][idx] / (prob_counts_df["{0}p_0a_lott".format(x)][idx]
                                                                                                + prob_counts_df["{0}p_0a_safe".format(x)][idx] 
                                                                                                + prob_counts_df["{0}p_0a_nr".format(x)][idx]))        
    risk_count_df = pd.DataFrame(risk_count) ## convert dicts to DFs
    risk_prop_df = pd.DataFrame(risk_prop)

    figure, axes = plt.subplots(2, 1, figsize=(14, 12))
    sns.despine(top = True)
    sns.barplot(risk_count_df, ax = axes[0])
    sns.barplot(risk_prop_df, ax = axes[1])
    axes[0].set_ylabel("Number of Trials")
    axes[0].set_xlabel("Lottery Probability/Response")
    axes[0].set_yticklabels(axes[0].get_yticks().astype(int))
    axes[0].bar_label(axes[0].containers[0])
    axes[1].set_ylim(0, 1)
    axes[1].set_ylabel("Proportion of Trials")
    axes[1].bar_label(axes[1].containers[0])
    axes[1].set_xlabel("Lottery Probability")
    plt.suptitle("RIC2 sub-{0} - Risk Trial Descriptives".format(subs[idx], fontsize = 20)) 
    save_sub_dir = os.path.join(base_proj_dir, "derivatives/RIC{0}/descriptives/crdm/sub-{1}".format(analysis, subs[idx]))
    fig_save = os.path.join(save_sub_dir, "sub-{0}_ric{1}-crdm_trials-risk.png".format(subs[idx], analysis)) 
    plt.savefig(fig_save)
    print("Saving to: {0}".format(fig_save)) 
    plt.show()

## Ambiguity Trials

In [ ]:
for idx, curr_sub in enumerate(subs):
    amb_count = {"24a_safe":[], "24a_lott":[], "24a_nr":[],
                "50a_safe":[], "50a_lott":[], "50a_nr":[],
                "74a_safe":[], "74a_lott":[], "74a_nr":[]}
    amb_prop = {"24a_lott":[], "50a_lott":[], "74a_lott":[],}
    for x in ["24", "50", "74"]:
        for y in ["safe", "lott", "nr"]:
            amb_count["{0}a_{1}".format(x, y)].append(amb_counts_df["50p_{0}a_{1}".format(x, y)][idx]) 
        amb_prop["{0}a_lott".format(x)].append(amb_counts_df["50p_{0}a_lott".format(x)][idx] / (amb_counts_df["50p_{0}a_lott".format(x)][idx]
                                                                                                + amb_counts_df["50p_{0}a_safe".format(x)][idx] 
                                                                                                + amb_counts_df["50p_{0}a_nr".format(x)][idx]))        
    amb_count_df = pd.DataFrame(amb_count)
    amb_prop_df = pd.DataFrame(amb_prop)

    figure, axes = plt.subplots(2, 1, figsize=(14, 12))
    
    sns.barplot(amb_count_df, ax = axes[0])
    sns.despine(top = True)
    axes[0].set_ylabel("Number of Trials")
    axes[0].set_xlabel("Lottery Probability/Response")
    axes[0].set_yticklabels(axes[0].get_yticks().astype(int))
    axes[0].bar_label(axes[0].containers[0])

    sns.barplot(amb_prop_df, ax = axes[1])
    axes[1].set_ylim(0, 1)
    axes[1].set_ylabel("Proportion of Trials")
    axes[1].bar_label(axes[1].containers[0])
    axes[1].set_xlabel("Lottery Probability")
    plt.suptitle("RIC2 sub-{0} - Ambiguity Trial Descriptives".format(subs[idx], fontsize = 20)) 
    save_sub_dir = os.path.join(base_proj_dir, "derivatives/RIC{0}/descriptives/crdm/sub-{1}".format(analysis, subs[idx]))
    fig_save = os.path.join(save_sub_dir, "sub-{0}_ric{1}-crdm_trials-amb.png".format(subs[idx], analysis)) 
    plt.savefig(fig_save)
    print("Saving to: {0}".format(fig_save)) 
    plt.show()

#### Use to remove files if I decide to change naming convention

In [ ]:
for idx, curr_sub in enumerate(subs):
    remove_dir = os.path.join(base_proj_dir, "derivatives/RIC{0}/descriptives/crdm/sub-{1}".format(analysis, subs[idx]))
    fig_remove = os.path.join(remove_dir, "sub-{0}_ric{1}-crdm_amb-trials.png".format(subs[idx], analysis))       
    #os.remove(fig_remove) ## uncomment only if you're **VERY** sure you want to delete something...
    print(curr_sub)